In [1]:
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
from STL_RS import results_analysis
from econ import economics
from functions import (
    lmtd,
    h_s_fg,
    fg_calculation,
    HX_calculation,
    cw_Tout,
    decision_variable_placement,
    Pressure_calculation,
    tur_comp_pratio,
    turbine_compressor_calculation,
    cooler_calculation,
    heater_calculation,
    hx_side_selection,
    h0_fg,
    s0_fg,
    hin_fg,
    sin_fg,
    h0,
    s0,
    T0,
    P0,
    K,
)

* The layouts from ML model is given in hot-encoded form with start(G) and end(E) tokens.
* The unit information from layouts are obtained by slicing the matrix from both ends.

In [2]:
layout = torch.tensor(
    [
        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ]
)
units = layout[1:-1]
print(units)

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])


* x is a list of decision variable values for each particle in the swarm to optimize
* Equipment information from the layout (units) and their respective decision variables are added to equipment list and bounds list
* hx_token in here is used to limit the number of Heat exchanger decision variables as the same HX is present 2 times in the layout
* The last bound is for the mass flow of sCO2 in the cycle

In [3]:
# x =[] x starts of as an empty list for each optimization but for demonstration purposes for this file,
# the values of base case are added to the list to help run the code for a particle 
x = [78.5e5,10.8,32.3,241.3e5,10.8,411.4,93.18]
equipment = np.zeros(len(units)).tolist()
bounds = list(range(len(units)))
hx_token = 1
for i in range(len(units)):
    a = np.where(units[i] == 1)[0][0]
    if a == 1:
        equipment[i] = 1
        bounds[i] = (74e5, 300e5)
    elif a == 2:
        equipment[i] = 2
        bounds[i] = (32, 38)
    elif a == 3:
        equipment[i] = 3
        bounds[i] = (74e5, 300e5)
    elif a == 4:
        equipment[i] = 4
        bounds[i] = (180, 530)
    elif a == 5:
        equipment[i] = 5
        if hx_token == 1:
            bounds[i] = (4, 11)
            hx_token = 0
        else:
            bounds[i] = (0, 0)
    elif a == 6:
        equipment[i] = 6
bounds.append((50, 160))
print(equipment)
print(bounds)

[1, 5, 2, 3, 5, 4]
[(7400000.0, 30000000.0), (4, 11), (32, 38), (7400000.0, 30000000.0), (0, 0), (180, 530), (50, 160)]


* The parameters of the PSO algorithm is set as desired
* Number of decision variables(nv) is also obtained for PSO algorithm

In [4]:
particle_size = 7 * len(bounds)
iterations = 30
nv = len(bounds)

* The parameters of the model comes from isentropic efficiencies, cooling water tempeerature, flue gas(fg) information from GT topping cycle and lastly pressure drop assumptions for equipment.
* A fix penalty value is placed to differentiate the infeasible solutions from feasible ones and stop further execution of optimization in any calculation step to save computational time.


In [5]:
ntur = 85  # turbine efficiency     2019 Nabil
ncomp = 82  # compressor efficiency 2019 Nabil
cw_temp = 19  # °C
fg_tin = 539  # °C
fg_m = 68.75  # kg/s
cooler_pdrop = 1e5
heater_pdrop = 0
hx_pdrop = 0.5e5
PENALTY_VALUE = float(1e6)

* Necessary arrays to hold the information for the whole system is created at the beginning of the optimization to avoid unnecessary complications.
* All the information related to the equipment is stored in the equipment position of each of the arrays.

In [6]:
enumerated_equipment = list(enumerate(equipment))
Temperatures = np.zeros(len(equipment))
Pressures = np.zeros(len(equipment))
enthalpies = np.zeros(len(equipment))
entropies = np.zeros(len(equipment))
exergies = np.zeros(len(equipment))
w_comp = np.zeros(len(equipment))
cost_comp = np.zeros(len(equipment))
comp_pratio = np.ones(len(equipment))
w_tur = np.zeros(len(equipment))
cost_tur = np.zeros(len(equipment))
tur_pratio = np.ones(len(equipment))
q_cooler = np.zeros(len(equipment))
cost_cooler = np.zeros(len(equipment))
dissipation = np.zeros(len(equipment))
q_heater = np.zeros(len(equipment))
cost_heater = np.zeros(len(equipment))
q_hx = np.zeros(len(equipment))
cost_hx = np.zeros(len(equipment))
print("(Equipment Position,Equipment Type) =",enumerated_equipment)

(Equipment Position,Equipment Type) = [(0, 1), (1, 5), (2, 2), (3, 3), (4, 5), (5, 4)]


* The decision variables, which are picked based on bounds, are matched with the corresponding equipments
* Turbines and compressors have their target pressures
* Heaters and coolers have their target temperatures
* Heat Exchangers have their DTmin or approach temperatures
* Splitters have their split ratios
* The layouts without any heat exchangers or splitters/mixers will get a default value of `1` which will not be used in the optimization


In [7]:
(
        Pressures,
        Temperatures,
        approach_temp,
        split_ratio,
        m,
    )= decision_variable_placement(x, enumerated_equipment, Pressures, Temperatures)
print("Pressures =", Pressures)
print("Temperatures =", Temperatures)
print("approach_temp =", approach_temp)
print("split_ratio =", split_ratio)
print("m =", m)

Pressures = [ 7850000.        0.        0. 24130000.        0.        0.]
Temperatures = [  0.    0.   32.3   0.    0.  411.4]
approach_temp = 10.8
split_ratio = 1
m = 93.18


* After the initalization of equipment with decision variables, pressures of all units can be calculated.
* The equipments have their fix pressure drops or outlet pressures which enables the calculation of the pressures for the following units if you know the previous one.

In [8]:
Pressures = Pressure_calculation(
        Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop
    )
print("Pressures =", Pressures)

Pressures = [ 7850000.  7800000.  7700000. 24130000. 24080000. 24080000.]
